# Auto_Price_Prediction - ML Model
-----------------------------
- Unversity of Solent : 
- BSc Final year Project :
- M.A.M Imamdeen

In [ ]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Reading and Displaying the dataset

In [17]:
df = pd.read_csv("Toyota_Vehicle_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018.0,"9,300,000.00",8000,Petrol
1,Toyota,Allion,2018.0,"9,775,000.00",9185,Petrol
2,Toyota,Allion,2018.0,"9,890,000.00",9875,Petrol
3,Toyota,Allion,2018.0,"9,850,000.00",10907,Petrol
4,Toyota,Allion,2008.0,"4,885,000.00",13000,Petrol
...,...,...,...,...,...,...
2332,Ford Endeavor 4x4 Thunder Plus,Ford,2016.0,NaN,NaN,NaN
2333,Tata Indica V2,Tata,2019.0,NaN,NaN,NaN
2334,NaN,NaN,NaN,NaN,NaN,NaN
2335,Maruti Suzuki Wagon R,Maruti,2014.0,"4,425,000",67000,Petrol


# Pre-Processing and Finding out Anomalies

In [9]:
df.shape

(2337, 6)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        2335 non-null   object 
 1   company     2335 non-null   object 
 2   year        2335 non-null   float64
 3   Price       2333 non-null   object 
 4   kms_driven  2333 non-null   object 
 5   fuel_type   2332 non-null   object 
dtypes: float64(1), object(5)
memory usage: 109.7+ KB


In [19]:
df["name"].unique()

array(['Toyota', 'Toyota Innova 2.5 E 8 STR',
       'Maruti Suzuki Alto 800 Vxi', 'Hyundai Accent 2001 Car',
       'Maruti Suzuki Omni', 'Mahindra KUV100', nan, 'Datsun Go Plus',
       'Ford Endeavor 4x4 Thunder Plus', 'Tata Indica V2',
       'Maruti Suzuki Wagon R ', 'Maruti Suzuki Swift Select Variant'],
      dtype=object)

In [20]:
df["name"].nunique()

11

### Anomalies found in "name" column:¶
 * Names are pretty inconsistent.
 * Names have company names attached to it.
 * Some names are spam like 'Maruti Ertiga showroom condition with', 'I want to sell my car Tata Zest', etc.
 * Consider first 2-3 words from the names as final name, in-order to make the categorical.

In [21]:
df["company"].unique()

array(['Allion', 'Allion ', 'Allion 240', 'Allion 240 ', 'Allion 260',
       'Aqua', 'Aqua G Grade', 'Aqua G grade', 'Aqua G Sport',
       'Aqua S Grade', 'Aqua X Urban', 'Avanza', 'Axio', 'Axio Fielder',
       'Axio G Grade', 'Axio WXB', 'Axio X Grade', 'Belta', 'Carina',
       'Corolla 110', 'Corolla 121', 'Corolla 141', 'Corona Premio',
       'IST', 'Passo', 'Pixis', 'Platz', 'Premio', 'Premio 240',
       'Premio 260', 'Premio G Superior', 'Prius', 'Prius 3rd Genaration',
       'Prius 3rd Generation', 'Prius S Grade', 'Prius S Touring',
       'Roomy', 'Soluna', 'soluna', 'Tank', 'Vios', 'Vitz',
       'Vitz Edition 2', 'Vitz Edition 3', 'Vitz F', 'Vitz KSP90',
       'Yaris', 'Toyota', 'Maruti', 'Hyundai', 'Mahindra', nan, 'Datsun',
       'Ford', 'Tata'], dtype=object)

In [22]:
df["company"].nunique()

54

### Anomalies found in "company" column:¶
* Some "company" names are incorrect.
* Ex:- 'URJENT', 'Used', '7', '2012', etc

In [24]:
df["year"].unique()

array([2018., 2008., 2017., 2016., 2014., 2013., 2011., 2010., 2009.,
       2007., 2012., 2006., 2004., 2003., 2002., 2001., 2005., 2015.,
       2019., 2000., 2020.,   nan])

In [25]:
df["year"].nunique()

21

# Anomalies found in "year" column:¶
* Many garbage/non-year values are present.
* Had to convert the dtype of the "year" column from object to int64.

In [26]:
df["Price"].unique()

array(['9,300,000.00', '9,775,000.00', '9,890,000.00', '9,850,000.00',
       '4,885,000.00', '8,000,000.00', '7,675,000.00', '7,900,000.00',
       '7,000,000.00', '8,900,000.00', '8,850,000.00', '8,150,000.00',
       '8,200,000.00', '7,600,000.00', '6,500,000.00', '5,735,000.00',
       '6,725,000.00', '5,650,000.00', '5,300,000.00', '6,600,000.00',
       '5,600,000.00', '5,750,000.00', '4,950,000.00', '5,500,000.00',
       '4,900,000.00', '5,450,000.00', '5,875,000.00', '5,525,000.00',
       '4,985,000.00', '4,450,000.00', '4,790,000.00', '4,575,000.00',
       '5,475,000.00', '5,100,000.00', '5,900,000.00', '4,625,000.00',
       '5,790,000.00', '6,200,000.00', '4,400,000.00', '4,350,000.00',
       '5,200,000.00', '5,390,000.00', '4,725,000.00', '4,750,000.00',
       '5,385,000.00', '5,325,000.00', '4,650,000.00', '3,825,000.00',
       '3,500,000.00', '4,590,000.00', '4,850,000.00', '4,895,000.00',
       '3,600,000.00', '3,275,000.00', '3,290,000.00', '3,390,000.00',
      

In [27]:
df["Price"].nunique()

549

In [28]:
count = 0
for each_price in df["Price"]:
    if each_price == "Ask For Price":
        count += 1
print("No. of records in df with Price as \"Ask For Price\":", count)

No. of records in df with Price as "Ask For Price": 1


### Anomalies found in "Price" column:
* Have to look after the "Ask for Price" type values in the column.
* Had to convert the dtype of the "Price" column from object to int64 by removing the commas in-between them.

In [29]:
df["kms_driven"].unique()

array(['8000', '9185', '9875', '10907', '13000', '22300', '26000',
       '27000', '31000', '34000', '41000', '47000', '49000', '54000',
       '56000', '60000', '71000', '71500', '75000', '76501', '85000',
       '86000', '87000', '89000', '90000', '90080', '92000', '92700',
       '93000', '96000', '97000', '100000', '102000', '104000', '105000',
       '106000', '107000', '109000', '110000', '112000', '112500',
       '113000', '115000', '116000', '117000', '118000', '120000',
       '121000', '124000', '125000', '126000', '128000', '130000',
       '133000', '135000', '137000', '138000', '140000', '142000',
       '144000', '145000', '146000', '147000', '152000', '153000',
       '156000', '160000', '168000', '170000', '170676', '180000',
       '190000', '192000', '200000', '1000', '32000', '69000', '82000',
       '95000', '99000', '102500', '107960', '122600', '127000', '132000',
       '143000', '151000', '152547', '161300', '163000', '165000',
       '171000', '185000', '18600

In [30]:
df["kms_driven"].nunique()

667

In [31]:
count = 0

for each_kms in df["kms_driven"]:
    if each_kms == "Petrol":
        count += 1
        
print("No. of records in df with kms_driven as \"nan\":", df["kms_driven"].isnull().sum())
print("No. of records in df with kms_driven as \"Petrol\":", count)

No. of records in df with kms_driven as "nan": 4
No. of records in df with kms_driven as "Petrol": 0


### Anomalies found in "kms_driven" column:
* Had to look after the "Petrol" type values in the column.
* Had to convert the dtype of the "kms_driven" column from object with "kms" in the end to int64 by removing the them.

In [32]:
df["fuel_type"].unique()

array(['Petrol', 'Hybrid', 'Electric', 'Diesel', nan], dtype=object)

In [33]:
df["fuel_type"].nunique()

4

In [34]:
print("\"fuel_type\" column has", df["fuel_type"].isnull().sum(), "null values.")

"fuel_type" column has 5 null values.


# Data Sanity (Clearing) & Filtering

### Converting the "year" column to _int64_ type

In [37]:
df["year"] = pd.to_numeric(df["year"])

### Removing all the records, having _"Price"_ == _"Ask For Price"_

In [38]:
df = df[df["Price"] != "Ask For Price"]

### Converting the "Price" column to pure numeric, by removing "," literals

In [39]:
df["Price"] = pd.to_numeric(df["Price"].str.replace(",", ""))

### Removing the _"kms"_ from "kms_driven" column and replacing "," literals with ""

In [40]:
df["kms_driven"] = df["kms_driven"].str.split().str.get(0).str.replace(",", "")

### Removing _null_ records from "fuel_type" column

In [43]:
df = df[~df["fuel_type"].isnull()]

### Considering only the first _3 words_ from the "name" column 

In [44]:
df["name"] = df["name"].str.split().str.slice(0, 3).str.join(" ")

### Resetting the indexes of the dataframe

In [45]:
df = df.reset_index(drop=True)

In [46]:
df

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018.0,9300000.0,8000,Petrol
1,Toyota,Allion,2018.0,9775000.0,9185,Petrol
2,Toyota,Allion,2018.0,9890000.0,9875,Petrol
3,Toyota,Allion,2018.0,9850000.0,10907,Petrol
4,Toyota,Allion,2008.0,4885000.0,13000,Petrol
...,...,...,...,...,...,...
2326,Hyundai Accent 2001,Hyundai,2001.0,1875000.0,75000,Petrol
2327,Maruti Suzuki Omni,Maruti,2011.0,1350000.0,135582,Petrol
2328,Datsun Go Plus,Datsun,2017.0,2750000.0,90000,Petrol
2329,Maruti Suzuki Wagon,Maruti,2014.0,4425000.0,67000,Petrol


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        2331 non-null   object 
 1   company     2331 non-null   object 
 2   year        2331 non-null   float64
 3   Price       2331 non-null   float64
 4   kms_driven  2331 non-null   object 
 5   fuel_type   2331 non-null   object 
dtypes: float64(2), object(4)
memory usage: 109.4+ KB


In [48]:
df.describe()

,year,Price
count,2331.000000,2.331000e+03
mean,2011.128700,4.557764e+06
std,5.046283,1.806871e+06
min,2000.000000,1.275000e+06
25%,2007.000000,3.375000e+06
50%,2012.000000,4.190000e+06
75%,2015.000000,5.225000e+06
max,2020.000000,1.260000e+07


In [49]:
df["Price"].min()

1275000.0

In [50]:
df["Price"].quantile(0.25)

3375000.0

In [51]:
df["Price"].median()

4190000.0

In [52]:
df["Price"].quantile(0.75)

5225000.0

In [53]:
df["Price"].max()

12600000.0

In [54]:
find_outlier = df[df["Price"].map(str).apply(len) > 6]

In [55]:
find_outlier["Price"].min()

1275000.0

In [56]:
find_outlier["Price"].quantile(0.25)

3375000.0

In [57]:
find_outlier["Price"].median()

4190000.0

In [58]:
find_outlier["Price"].quantile(0.75)

5225000.0

In [59]:
find_outlier["Price"].max()

12600000.0

In [60]:
find_outlier[find_outlier["Price"] > find_outlier["Price"].quantile(0.75)]

,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018.0,9300000.0,8000,Petrol
1,Toyota,Allion,2018.0,9775000.0,9185,Petrol
2,Toyota,Allion,2018.0,9890000.0,9875,Petrol
3,Toyota,Allion,2018.0,9850000.0,10907,Petrol
5,Toyota,Allion,2017.0,8000000.0,22300,Petrol
...,...,...,...,...,...,...
2285,Toyota,Vitz F,2018.0,5495000.0,11500,Petrol
2286,Toyota,Vitz F,2018.0,5750000.0,15000,Petrol
2307,Toyota,Yaris,2020.0,7200000.0,500,Petrol
2308,Toyota,Yaris,2020.0,6850000.0,2200,Petrol


In [68]:
outlier = find_outlier[find_outlier["Price"] == 9775000]
outlier

,name,company,year,Price,kms_driven,fuel_type
1,Toyota,Allion,2018.0,9775000.0,9185,Petrol
1424,Toyota,Premio G Superior,2018.0,9775000.0,17200,Petrol
1454,Toyota,Premio G Superior,2017.0,9775000.0,66000,Petrol


In [69]:
df = df[df["Price"] != 9775000]

In [70]:
df


,name,company,year,Price,kms_driven,fuel_type
0,Toyota,Allion,2018.0,9300000.0,8000,Petrol
2,Toyota,Allion,2018.0,9890000.0,9875,Petrol
3,Toyota,Allion,2018.0,9850000.0,10907,Petrol
4,Toyota,Allion,2008.0,4885000.0,13000,Petrol
5,Toyota,Allion,2017.0,8000000.0,22300,Petrol
...,...,...,...,...,...,...
2326,Hyundai Accent 2001,Hyundai,2001.0,1875000.0,75000,Petrol
2327,Maruti Suzuki Omni,Maruti,2011.0,1350000.0,135582,Petrol
2328,Datsun Go Plus,Datsun,2017.0,2750000.0,90000,Petrol
2329,Maruti Suzuki Wagon,Maruti,2014.0,4425000.0,67000,Petrol


In [74]:
df.to_csv("Cleaned_Data.csv", index=False)

In [75]:
X = df.drop(columns = "Price")
y = df["Price"]

In [76]:
X

,name,company,year,kms_driven,fuel_type
0,Toyota,Allion,2018.0,8000,Petrol
2,Toyota,Allion,2018.0,9875,Petrol
3,Toyota,Allion,2018.0,10907,Petrol
4,Toyota,Allion,2008.0,13000,Petrol
5,Toyota,Allion,2017.0,22300,Petrol
...,...,...,...,...,...
2326,Hyundai Accent 2001,Hyundai,2001.0,75000,Petrol
2327,Maruti Suzuki Omni,Maruti,2011.0,135582,Petrol
2328,Datsun Go Plus,Datsun,2017.0,90000,Petrol
2329,Maruti Suzuki Wagon,Maruti,2014.0,67000,Petrol


In [77]:
y

0       9300000.0
2       9890000.0
3       9850000.0
4       4885000.0
5       8000000.0
          ...    
2326    1875000.0
2327    1350000.0
2328    2750000.0
2329    4425000.0
2330    5425000.0
Name: Price, Length: 2328, dtype: float64

In [78]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [79]:
ohenc = OneHotEncoder()
ohenc.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [80]:
column_trans = make_column_transformer((OneHotEncoder(categories = ohenc.categories_), ['name','company','fuel_type']), remainder='passthrough')

In [81]:
lnregr = LinearRegression()

In [82]:
final = make_pipeline(column_trans, lnregr)

In [83]:
final.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Datsun Go Plus', 'Hyundai Accent 2001', 'Maruti Suzuki Omni',
       'Maruti Suzuki Swift', 'Maruti Suzuki Wagon', 'Toyota',
       'Toyota Innova 2.5'], dtype=object),
                                                                            array(['Allion', 'Allion ', 'Allion 240', 'Allion 240 ', 'Allion 260',
       'Aqua', 'Aq...
       'Premio', 'Premio 240', 'Premio 260', 'Premio G Superior', 'Prius',
       'Prius 3rd Genaration', 'Prius 3rd Generation', 'Prius S Grade',
       'Prius S Touring', 'Roomy', 'Soluna', 'Tank', 'Toyota', 'Vios',
       'Vitz', 'Vitz Edition 2', 'Vitz Edition 3', 'Vitz F', 'Vitz KSP90',
       'Yaris', 'soluna'], dtype=object),
                                                                            ar

In [84]:
final.score(X_test, y_test)

0.8864485673137213

In [85]:
y_pred = final.predict(X_test)

In [86]:
y_pred

array([4753771.34142625, 3598827.45470881, 5417106.38745368,
       5547102.95082766, 3656439.71826702, 3168217.18145502,
       2835700.571271  , 4379142.77704734, 2190239.12727082,
       4593780.23395622, 5434410.82712042, 5131182.51984394,
       4979036.28697157, 3675220.92920882, 3849782.86356628,
       3711390.93535465, 1118149.77845317, 8766423.53993917,
       5260513.49161679, 4446703.85334086, 5240368.49596757,
       2583731.93758357, 2533889.34510183, 4023402.29181087,
       5588908.25095469, 7186856.24477214, 5059273.44059706,
       3354241.66078222, 5246908.30351746, 1841435.82039332,
       6553202.49358249, 3758357.24867791, 5026270.1042847 ,
       2324127.02409321, 3290352.86403048, 9209438.35116649,
       3672265.49802762, 8802672.69810772, 8574208.27955389,
       2569167.23411602, 5254566.17422169, 3557769.55754614,
       2743238.67590916, 3390038.04899395, 3081441.07763082,
       4880062.37743473, 5328155.73754287, 2681151.55016154,
       5370372.55906779,

In [87]:
r2_score(y_test, y_pred)

0.8864485673137213

In [88]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    print(r2_score(y_test, y_pred))

0.8873803472376742
0.8925378568421105
0.8780760125315085
0.8818907775986817
0.8802224449778788
0.8423417109966314
0.8905257170957597
0.8708075676826071
0.8944100462163831
0.8723376240113345


In [ ]:
scores = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = i)
    
    lnregr = LinearRegression()
    
    final = make_pipeline(column_trans, lnregr)
    
    final.fit(X_train, y_train)
    
    y_pred = final.predict(X_test)
    
    scores.append(r2_score(y_test, y_pred))

In [ ]:
np.argmax(scores)